In [106]:
import pandas as pd
from tqdm import tqdm

In [126]:
import os

In [122]:
import urllib.request
import re
import argparse
import sys
from bs4 import BeautifulSoup
import pandas as pd
import unittest


def f(x):
    return x[0] + x[1] + x[2]


def create_request(corpus, request, case):
    url = 'http://search2.ruscorpora.ru/search.xml?env=alpha&mycorp=&mysent=&mysize=&mysentsize=&mydocsize=&dpp=&spp=&spd=&' \
          'text=lexform&mode=%s&sort=gr_tagging&lang=ru&nodia=1&req=%s&gramm1=%s&sem1=&sem-m' \
          'od1=sem&sem-mod1=sem2&flags1=&m1=&parent2=0&level2=0&min2=1&max2=1&lex2=&gramm2=&sem2=&sem-mod2=sem&se' \
          'm-mod2=sem2&flags2=&m2=&out=%s'
    common_url = url % (corpus, request, case, 'kwic')
    return common_url


def get_all_pages(common_url, results):
    pages = results // 10
    k = 0
    massive_of_links = []
    while k < pages:
        page = common_url + '&p=' + str(k)
        massive_of_links.append(page)
        k += 1
    return massive_of_links


def get_table(urls, n_results, write, kwic):
    center_right_list = []
    gram_list =[]
    center_list = []
    right_list = []
    left_list = []
    for url in urls:
        soup_url = urllib.request.urlopen(url)
        soup = BeautifulSoup(soup_url, 'lxml')
        for center_right in soup.select('tr > td > nobr'):
            center_right_list.append(center_right.text)
        for left in soup.select('tr > td > div > nobr'):
            left_list.append(left.text)
    center_list = center_right_list[0::2]
    right_list = center_right_list[1::2]
    if n_results == '':
        n_results = int(len(right_list))
    d = {"center": center_list[:n_results], "right": right_list[:n_results], "left": left_list[:n_results]}
    s = pd.DataFrame(d, columns=["left", "center", "right"])
    if write is True:
        file = open('ru_table.csv', 'w',encoding='utf-8-sig')
        s.to_csv(file, sep=';')
        file.close()
    else:
        pass
    if kwic is False:
        file = open('ru_table.csv', 'w', encoding='utf-8-sig')
        s = s.apply(f, axis=1)
        s.to_csv(file, sep=';')
    else:
        pass
    if s.empty:
        pass
        #print('Something went wrong: please try again!')
    else:
        return s

def main(query, corpus='main', tag='', n_results=10, write=False, kwic=True):
    
    request = urllib.request.quote(query.encode('windows-1251'))
    case = urllib.request.quote(tag.encode('windows-1251'))
    
    common_ur = create_request(corpus, request, case)
    #print(common_ur,'сайт')
    return get_table(get_all_pages(common_ur, n_results), n_results, write, kwic)
    

if __name__ == "__main__":
    args = sys.argv[1:]
    parser = argparse.ArgumentParser()
    parser.add_argument('corpus', type=str)
    parser.add_argument('query', type=str)
    parser.add_argument('tag', type=str)
    parser.add_argument('n_results', type=int)
    parser.add_argument('write', type=bool)
    parser.add_argument('kwic', type=bool)
    args = parser.parse_args(args)
    main(query, corpus, tag, n_results, write, kwic)

usage: ipykernel_launcher.py [-h] corpus query tag n_results write kwic
ipykernel_launcher.py: error: the following arguments are required: query, tag, n_results, write, kwic


SystemExit: 2

In [128]:
positive = pd.read_csv('pos_hypernymy.csv')

In [129]:
positive['child_context'] = ''
positive['parent_context'] = ''

In [137]:
example = type(lines)

In [136]:
type(lines)

pandas.core.series.Series

In [130]:
positive.head()

,child_id,parent_id,child_context,parent_context
0,КРЕСТНЫЙ ОТЕЦ,КРЕСТНЫЙ РОДИТЕЛЬ,,
1,ОБМАН ПОКУПАТЕЛЕЙ,НАРУШЕНИЕ ПРАВИЛ ТОРГОВЛИ,,
2,ВОЕННАЯ ТАКТИКА,"ТАКТИКА, ТАКТИЧЕСКИЕ ДЕЙСТВИЯ",,
3,УСМЕШКА,УЛЫБКА,,
4,РЕЦЕПТУРА ЛЕКАРСТВА,РЕЦЕПТ ПРИГОТОВЛЕНИЯ,,


In [138]:
counter = 0
for i in tqdm(range(len(positive.child_id)), desc='Progress'):
    lines = main(positive.child_id[i].lower(),n_results=14, kwic=False, write=False)
    if type(lines) == example:
        line = lines[0].replace('\xa0\xa0←…→\n   ','').replace('\xa0','')
        positive.loc[i,'child_context'] = line
        print(line)






Progress:   0%|          | 0/37249 [00:00<?, ?it/s]C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


его: «А разве Авель Енукидзе, крестный отец твоей жены, был плох







Progress:   0%|          | 1/37249 [00:00<4:29:21,  2.30it/s]

за продажу некачественных товаров и обман покупателей могут вырасти в несколько







Progress:   0%|          | 2/37249 [00:00<4:16:11,  2.42it/s]

...

 Перегонка жидкостей на две фракции называлась







Progress:  31%|███       | 11443/37249 [1:47:23<5:05:57,  1.41it/s]




Progress:  31%|███       | 11444/37249 [1:47:23<4:14:43,  1.69it/s]

есть, а то ли нет, выделать этот телячий дар с помощью







Progress:  31%|███       | 11445/37249 [1:47:24<4:19:52,  1.65it/s]




Progress:  31%|███       | 11446/37249 [1:47:24<3:37:48,  1.97it/s]

KeyboardInterrupt: 

In [144]:
positive.to_csv('childs.csv')

In [116]:
example = pd.DataFrame()

In [117]:
type(main(positive.child_id[i].lower(),n_results=14, kwic=False, write=False)) != type(example)

Something went wrong: please try again!


True

In [104]:
main(positive.child_id[0].lower(),n_results=14, kwic=False, write=False)[0].replace('\xa0\xa0←…→\n   ','').replace('\xa0','')

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


'его: «А разве Авель Енукидзе, крестный отец твоей жены, был плох'

In [124]:
positive.head()

,child_id,parent_id,child_context,parent_context
0,КРЕСТНЫЙ ОТЕЦ,КРЕСТНЫЙ РОДИТЕЛЬ,"его: «А разве Авель Енукидзе, крестный отец тв...",
1,ОБМАН ПОКУПАТЕЛЕЙ,НАРУШЕНИЕ ПРАВИЛ ТОРГОВЛИ,за продажу некачественных товаров и обман поку...,
2,ВОЕННАЯ ТАКТИКА,"ТАКТИКА, ТАКТИЧЕСКИЕ ДЕЙСТВИЯ","Военная тактика ― одно, а житейская ― совсем",
3,УСМЕШКА,УЛЫБКА,Кривая усмешка шевельнула усики на губе Виталии,
4,РЕЦЕПТУРА ЛЕКАРСТВА,РЕЦЕПТ ПРИГОТОВЛЕНИЯ,,


In [127]:
positive.to_csv(os.getcwd()+"\\with_childs.csv" , encoding='utf-8-sig', header=True, index=False)

In [ ]:
   http://search2.ruscorpora.ru/search.xml?env=alpha&mycorp=&mysent=&mysize=&mysentsize=&mydocsize=&dpp=&spp=&spd=&text=lexform&mode=main&sort=gr_tagging&lang=ru&nodia=1&req=%D0%BA%D1%80%D0%B5%D1%81%D1%82%D0%BD%D1%8B%D0%B9+%D0%BE%D1%82%D0%B5%D1%86
http://processing.ruscorpora.ru/search.xml?env=alpha&mycorp=&mysent=&mysize=&mysentsize=&mydocsize=&dpp=&spp=&spd=&text=lexform&mode=main&sort=gr_tagging&lang=ru&nodia=1&req=%D0%BA%D1%80%D0%B5%D1%81%D1%82%D0%BD%D1%8B%D0%B9+%D0%BE%D1%82%D0%B5%D1%86
http://processing.ruscorpora.ru/search.xml?env=alpha&mycorp=&mysent=&mysize=&mysentsize=&mydocsize=&dpp=&spp=&spd=&text=lexform&mode=main&sort=i_grtagging&lang=ru&nodia=1&req=%D0%BA%D1%80%D0%B5%D1%81%D1%82%D0%BD%D1%8B%D0%B9+%D0%BE%D1%82%D0%B5%D1%86
   http://search2.ruscorpora.ru/search.xml?env=alpha&mycorp=&mysent=&mysize=&mysentsize=&mydocsize=&dpp=&spp=&spd=&text=lexform&mode=main&sort=gr_tagging&lang=ru&nodia=1&req=%EA%F0%E5%F1%F2%ED%FB%E9%20%EE%F2%E5%F6&gramm1=&sem1=&sem-mod1=sem&sem-mod1=sem2&flags1=&m1=&parent2=0&level2=0&min2=1&max2=1&lex2=&gramm2=&sem2=&sem-mod2=sem&sem-mod2=sem2&flags2=&m2=&out=kwic